In [9]:
import pandas as pd
import polars as pl

In [16]:
df = pd.read_html("https://en.wikipedia.org/wiki/List_of_cities_by_GDP")[0]

In [56]:
(
    pl.DataFrame(df)
    .filter(
        pl.col('Country/region').is_in(['United States','Canada'])
    )
    .select(
        pl.col('City proper/metropolitan area').alias('city'),
        pl.col('Country/region').alias('country'),
        pl.col('Official est. GDP up to date (billion US$)').str.split(' ').list[0].str.split(' ').list[0].str.replace(',','').cast(float).alias('gdp'),
        pl.col('Metropolitan population').str.split(' ').list[0].str.split('	').list[0].str.split('[').list[0].str.replace(',','').str.replace(',','').cast(float).alias('population')
    )
    .with_columns(
        z_gdp=(pl.col('gdp').log10() - 2.41108) / 0.41621,
        z_pop=(pl.col('population').log10() - 6.53039) / 0.33281
    )
    .with_columns(
        market=0.5 + 0.1*(0.35*pl.col('z_gdp') + 0.65*pl.col('z_pop'))
    )
    .with_columns(
        m_plus=pl.col('market')*0.6021
    )
    .with_columns(
        tru=146946*(7.5268*pl.col('m_plus')).exp() / 1000000
    )
    .to_pandas()
)

,city,country,gdp,population,z_gdp,z_pop,market,m_plus,tru
0,"Abbotsford, British Columbia",Canada,6.239,203907.0,-3.882571,-3.668633,0.125649,0.075653,0.259691
1,"Abilene, TX MSA",United States,9.469,179308.0,-3.447240,-3.836394,0.129981,0.078262,0.264840
2,"Akron, OH MSA",United States,44.562,697627.0,-1.831084,-2.063540,0.301782,0.181703,0.576922
3,"Albany-Lebanon, OR MSA",United States,6.108,130467.0,-3.904714,-4.251343,0.086998,0.052381,0.217964
4,"Albany-Schenectady-Troy, NY MSA",United States,80.303,904617.0,-1.216569,-1.724483,0.345329,0.207922,0.702788
...,...,...,...,...,...,...,...,...,...
411,"Yakima, WA MSA",United States,13.038,257001.0,-3.113498,-3.366651,0.172195,0.103679,0.320677
412,"York-Hanover, PA MSA",United States,25.187,461058.0,-2.426428,-2.603992,0.245816,0.148006,0.447678
413,"Youngstown-Warren-Boardman, OH-PA MSA",United States,24.990,535499.0,-2.434621,-2.408676,0.258224,0.155477,0.473574
414,"Yuba City, CA MSA",United States,8.608,182813.0,-3.546714,-3.811132,0.128141,0.077154,0.262641


In [15]:
df

city,country,gdp,population
str,str,list[str],list[str]
"""A Coruña metropolitan area""","""Spain""","[""28.819"", ""(2020)[2]""]","[""1,121,815"", ""(2020)[3]""]"
"""Aachen""","""Germany""","[""24.296"", ""(2020)[2]""]","[""249,070"", ""(2021)[4]""]"
"""Aalborg""","""Denmark""","[""31.855"", ""(2021)[2]""]","[""219,487"", ""(2021)[5]""]"
"""Aarhus""","""Denmark""","[""54.927"", ""(2021)[2]""]","[""352,751"", ""(2021)[6]""]"
"""Abbotsford, British Columbia""","""Canada""","[""6.239"", ""(2020)[7]""]","[""203,907"", ""(2020)[8]""]"
…,…,…,…
"""Zibo""","""China""","[""53.260"", ""(2020)[25][117]""]","[""3,525,000"", ""(2020)[12]""]"
"""Zürich Metropolitan Area""","""Switzerland""","[""158.913"", ""(2020)[136]""]","[""1,430,000"", ""(2020)[12]""]"
"""Zwolle""","""Netherlands""","[""17.936"", ""(2020)[2]""]","[""129,840"", ""(2021)[423]""]"
